In [3]:
import pystac_client 
import planetary_computer

catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",
                                    modifier=planetary_computer.sign_inplace,
)

In [4]:
time_range = "2000-02-24T00:00:00Z/2025-07-14T23:59:59Z"
bbox = [3.82715, 2.29382, 14.36961, 15.00814]
collection = "modis-11A1-061"

search = catalog.search(collections = [collection], bbox=bbox, datetime=time_range)
items = search.get_all_items()
len(items)

/Users/lshms102/miniconda3/envs/MPCP_lassasentinel/lib/python3.12/site-packages/pystac_client/item_search.py:896: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


70742

In [61]:
items.items[0].__dict__

{'links': [<Link rel=collection target=https://planetarycomputer.microsoft.com/api/stac/v1/collections/modis-11A1-061>,
  <Link rel=parent target=https://planetarycomputer.microsoft.com/api/stac/v1/collections/modis-11A1-061>,
  <Link rel=root target=<Client id=microsoft-pc>>,
  <Link rel=self target=https://planetarycomputer.microsoft.com/api/stac/v1/collections/modis-11A1-061/items/MYD11A1.A2025177.h19v08.061.2025178202641>,
  <Link rel=preview target=https://planetarycomputer.microsoft.com/api/data/v1/item/map?collection=modis-11A1-061&item=MYD11A1.A2025177.h19v08.061.2025178202641>],
 'stac_extensions': ['https://stac-extensions.github.io/projection/v2.0.0/schema.json',
  'https://stac-extensions.github.io/raster/v1.1.0/schema.json'],
 'id': 'MYD11A1.A2025177.h19v08.061.2025178202641',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[10.1148, 10.0014],
    [20.3173, 9.9999],
    [20.0099, -0.0015],
    [9.9628, 0],
    [10.1148, 10.0014]]]},
 'bbox': [9.9628, -0.0015, 20.3173, 

In [ ]:
mod11a1_items = [item for item in items if item.id.startswith("MOD11A1")]

In [32]:
import geopandas

df = geopandas.GeoDataFrame.from_features(mod11a1_items, crs="epsg:4326")

In [33]:
import numpy as np
np.shape(df)

(36978, 16)

In [72]:
# each STAC item has one or more assets, which includes links to the actual data files
import rich.table

table = rich.table.Table("Asset Key", "Description")
for asset_key, asset in mod11a1_items[0].assets.items():
    table.add_row(asset_key, asset.title)

table

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key        ┃ Description                                                         ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ hdf              │ Source data containing all bands                                    │
│ QC_Day           │ Quality control for daytime LST and emissivity                      │
│ Emis_31          │ Band 31 emissivity                                                  │
│ Emis_32          │ Band 32 emissivity                                                  │
│ QC_Night         │ Quality control for nighttime LST and emissivity                    │
│ metadata         │ Federal Geographic Data Committee (FGDC) Metadata                   │
│ LST_Day_1km      │ Daily daytime 1km grid Land-surface Temperature                     │
│ Clear_day_cov    │ Day clear-sky coverage                                              │
│ Day_view_angl    │ View zenith angle of daytime Landsurface Temperature                │
│ Day_view_time    │ (local solar) Time of daytime Land-surface Temperature observation  │
│ LST_Night_1km    │ Daily nighttime 1km grid Land-surface Temperature                   │
│ Clear_night_cov  │ Night clear-sky coverage                                            │
│ Night_view_angl  │ View zenith angle of nighttime Landsurface Temperature              │
│ Night_view_time  │ (local solar) Time of nighttime Landsurface Temperature observation │
│ tilejson         │ TileJSON with default rendering                                     │
│ rendered_preview │ Rendered preview                                                    │
└──────────────────┴─────────────────────────────────────────────────────────────────────┘

In [73]:
mod11a1_items[0].assets["rendered_preview"].to_dict()

{'href': 'https://planetarycomputer.microsoft.com/api/data/v1/item/preview.png?collection=modis-11A1-061&item=MOD11A1.A2025177.h19v08.061.2025178101459&assets=LST_Day_1km&tile_format=png&colormap_name=jet&rescale=255%2C310&format=png',
 'type': 'image/png',
 'title': 'Rendered preview',
 'rel': 'preview',
 'roles': ['overview']}

In [ ]:
from IPython.display import Image

Image(url=mod11a1_items[0].assets["rendered_preview"].href, width=500)